# 2 - Sudoku 

  O objetivo é preecncher uma grelha de *N²*$\times$*N²* com inteiros positivos no intervalo de 1 até *N²*, satisfazendo as seguintes regras:

  *   Cada inteiro no intervalo de 1 até *N²* ocorre só uma vez em cada coluna, linha e secção *N*$\times$*N*.
  *   No início do jogo uma fracção *0* $\leq \alpha <$ 1 das N⁴ casas da grelha são preenchidas de forma consistente com a regra anterior.

Pretende-se: 

  *   Construir um programa para inicializar a grelha a partir dos parâmetros N e $\alpha$.
  *   Construir soluções do problema paras as combinações de parâmeteos *N* $\in$ {3,4,5,6} e $\alpha \in$ {0.0, 0.2, 0.4, 0.6} e tirar conclusões da complexidade computacional destas soluções.

**Análise do Problema** 

  É esperado receber um *N* e um $\alpha$, por parte do utilizador, onde *N* corresponde ao tamanho de cada secção do Sudoku que será representado e $\alpha$ a fracção de casas já preenchidas pela máquina. 

  Deste modo, o método utilizado para a construção do Sudoku foi a de um grafo. A estrutura do grafo tem a seguinte forma, *(n, A)*, onde *n* corresponde ao nodo e *A* corresponde há lista de nodos que têm uma relação com ele, ou seja, 
pertemcem à mesma secção, linha ou coluna. 

Seguidamente, a função **ip_color2** que verifica todas as relações entre os nodos e atribui uma cor diferente para nodos adjacentes. Deste modo, não é possivel existir a mesma cor numa mesma secção, linha ou coluna. 

Finalmente, como cada cor é lhe atribuído um número $\varkappa \in$*N²*. O grafo é representado em forma de matriz com cada cor do nodo a representado como um número.    










In [1]:
!pip install ortools #não é necessário executar sempre

     |████████████████████████████████| 14.5 MB 85 kB/s 
     |████████████████████████████████| 1.1 MB 57.6 MB/s 
     |████████████████████████████████| 132 kB 76.8 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.12.0
    Uninstalling absl-py-0.12.0:
      Successfully uninstalled absl-py-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.2.0 requires absl-py<0.13,>=0.9, but you have absl-py 0.15.0 which is incompatible.


In [30]:
# importe de bibliotecas importantes para a execução do código
import numpy as np
import math 
import networkx as nx
import time
import random 
from ortools.linear_solver import pywraplp

In [3]:
def checkSquare(x1,y1,x2,y2,size):
        if x1//size == x2//size and y1//size == y2//size:
            return True
        return False

def createEdges(mat):
    adj=[]
    for y in range(len(mat)):
        for x in range(len(mat[y])):
            for outros in range(x,len(mat[y])):
                adj.append((mat[y][x],mat[y][outros]))
    for y in range(len(mat)):
        for x in range(len(mat[x])):
            for outros in range(y,len(mat)):
                adj.append((mat[y][x],mat[outros][x]))
            
    for y in range(len(mat)):
        for x in range(len(mat[y])):
            for yy in range(len(mat)):
                for xx in range(len(mat[yy])):
                    if checkSquare(x,y,xx,yy,int(math.sqrt(len(mat)))) and (mat[y][x],mat[yy][xx]) not in adj:
                        adj.append((mat[y][x],mat[yy][xx]))
    '''
    for (x,y) in adj:
        if (y,x) is adj:
            adj.remove((y,x))
    adj = list(set(adj))
    for (x,y) in adj:
        if x==y :
            adj.remove((x,y))'''
    adjq = [(x,y) for (x,y) in adj if x!=y and (y,x)]
    return adjq

In [4]:
def ip_color2(graph,k):
    

    # criar solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    #criar dicionario de variaveis x{i,j} 
    x = {}
    for i in graph:
        x[i] = {}
        for j in range(k):
            x[i][j] = solver.IntVar(0,1,'x[%i][%i]' % (i,j))

    my_list = []
      
    # vertices adjacentes tem cores diferentes
    for o in graph:
      for d in graph[o]:
            for j in range(k):
              if my_list.count(d) < 1:
                 solver.Add(x[o][j] + x[d][j] <= 1)
      my_list.append(o)  
         
         
       
                 


           
                  
            
    
    # cada vertice adjacente tem cores diferentes

    for i in graph:
        solver.Add(sum([x[i][j] for j in range(k)]) == 1)  # ou solver.Add(sum(list(x[i].values())) == 1) 

    # invocar solver e colorir o grafo

    stat = solver.Solve()
    if stat == pywraplp.Solver.OPTIMAL:

      # colorir

        for i in graph:
            for j in range(k):
                if round(x[i][j].solution_value()) == 1:
                    graph.nodes[i]['color'] = j+1
        return True
    else:
        return False

In [41]:
def converte(grafo, size, prob):
    cores=[]
    value = randint(0, size)
    for i in grafo:
        cores.append(grafo.nodes[i]['color'])

    for i in range(len(cores)):
        cores[i] = ((cores[i] + value) % size) + 1
    solvedSudoku=np.reshape(cores,(int(math.sqrt(len(cores))),int(math.sqrt(len(cores)))))

    solvedSudoku = apagaMat(solvedSudoku, prob)
    return solvedSudoku

In [40]:
def probCheck(prob):
    n = random.random()
    if(n <= prob):
        return True 
    return False 

def apagaMat(mat, prob): 
    prob = 1 - prob
    for y in range(len(mat)): 
        for x in range(len(mat[y])): 
            if probCheck(prob): 
                mat[y][x] = 0
    return mat 

Execução do código para n = 3.


In [47]:
pontos = [x+1 for x in range(9**2)]
pontos = np.reshape(pontos,(9,9))
edges=createEdges(pontos)
grafo5=nx.Graph(edges)
nx.set_node_attributes(grafo5,0,'color')
ti = time.perf_counter()
ip_color2(grafo5,9)
tf = time.perf_counter()
print(converte(grafo5, 9, 0.6))
print(f"Time {tf - ti: 0.5f} seconds")


[[0 6 7 0 0 5 3 2 8]
 [8 0 0 1 3 2 0 6 0]
 [2 0 3 6 8 7 4 1 0]
 [4 0 9 0 0 0 0 0 3]
 [0 0 2 9 5 4 1 8 0]
 [0 8 0 0 0 3 2 9 4]
 [7 3 8 0 6 0 0 0 0]
 [0 4 6 0 0 1 0 3 2]
 [0 0 0 3 4 8 6 0 9]]
Time  8.24172 seconds


Execução do código para n = 4.

In [48]:
pontos = [x+1 for x in range(16**2)]
pontos = np.reshape(pontos,(16,16))
edges=createEdges(pontos)
grafo5=nx.Graph(edges)
nx.set_node_attributes(grafo5,0,'color')
ti = time.perf_counter()
ip_color2(grafo5,16)
tf = time.perf_counter()
print(converte(grafo5, 16, 0.4))
print(f"Time {tf - ti: 0.5f} seconds")

[[ 0  0  0  0  0  0 10  0 12  0 14 15 16  1  0  0]
 [ 8  0 10  0  4  5  6  7  0  1  0  0 12 13 14  0]
 [12  0  0  0 16  0  2  0  0  5  0  0  8  0 10  0]
 [ 0  0  2  0  0  0 14  0  0  9 10 11  4  5  0  7]
 [ 0  0  0  0  0  0  0 10 13 12 15  0  1  0  3  2]
 [ 9  8  0 10  0  0  7  0  0 16  0  2  0  0  0 14]
 [13  0  0  0  0 16  0  0  0  0  0  0  9  8 11 10]
 [ 1  0  0  2  0  0  0  0  9  0 11  0  5  4  7  0]
 [ 0  0  0  5  0  0  0  0 14  0 12  0  2  3  0  0]
 [ 0 11  0  0  0  7  0  0  0  3  0  1  0  0 12 13]
 [14  0  0  0  0  3  0  0  0  7  0  0 10 11  0  0]
 [ 0  3  0  0 14 15  0  0  0  0  8  0  6  0  0  0]
 [ 7  0  0  0  0 10  9  0  0  0 13 12  0  2  0  0]
 [ 0  0  0  0  7  0  5  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 12  3  2  1 16  7  0  5  4  0 10  0  0]
 [ 3  0  1 16 15  0 13  0  0  0  9  0  0  0  0  4]]
Time  3.02827 seconds


Execução do código para n = 5.

In [49]:
pontos = [x+1 for x in range(25**2)]
pontos = np.reshape(pontos,(25,25))
edges=createEdges(pontos)
grafo5=nx.Graph(edges)
nx.set_node_attributes(grafo5,0,'color')
ti = time.perf_counter()
ip_color2(grafo5,25)
tf = time.perf_counter()
print(converte(grafo5, 25, 0.4))
print(f"Time {tf - ti: 0.5f} seconds")

[[4 4 4 0 4 4 4 4 0 0 4 4 4 4 0 4 0 4 4 4 0 0 4 0 0]
 [0 0 0 0 0 0 4 0 0 0 0 0 4 4 0 0 0 0 4 0 0 0 4 4 4]
 [0 4 4 0 0 0 4 4 0 0 0 0 0 0 0 4 0 0 0 0 4 0 4 4 4]
 [0 4 0 0 4 0 4 0 0 0 0 4 0 4 4 0 0 0 0 0 0 4 0 0 0]
 [4 0 4 0 4 0 0 4 0 4 4 0 0 0 4 0 0 0 0 0 0 4 0 4 0]
 [4 0 0 0 0 0 0 4 0 4 0 0 0 4 0 0 0 0 0 4 0 4 4 4 0]
 [4 4 4 4 0 0 0 0 0 0 0 4 0 4 0 4 0 0 4 0 4 4 4 4 4]
 [0 4 0 0 0 0 4 0 0 0 0 4 4 0 4 0 0 4 0 4 4 0 0 4 0]
 [4 0 0 4 0 0 4 0 0 4 0 0 0 0 4 0 0 4 0 0 4 4 0 0 4]
 [4 0 4 4 4 0 4 4 0 0 0 0 4 0 0 4 4 4 0 4 0 0 0 0 4]
 [0 0 0 4 0 0 4 4 0 0 0 0 4 0 4 4 0 0 4 0 0 0 0 0 4]
 [4 4 4 0 0 4 0 4 4 4 0 0 0 4 0 0 0 0 4 4 0 4 0 4 0]
 [0 0 0 0 0 0 0 0 0 4 0 4 0 4 4 0 4 0 0 0 4 0 4 0 4]
 [0 4 4 4 4 0 0 0 4 0 0 0 0 4 0 0 4 4 4 4 0 4 4 0 4]
 [0 0 4 4 0 0 0 4 0 0 0 0 4 0 4 0 4 0 0 0 4 0 0 4 0]
 [0 0 4 0 4 4 0 4 4 0 4 4 0 0 4 4 4 4 4 4 0 0 4 4 4]
 [0 0 4 0 0 4 0 0 4 0 4 0 4 0 0 0 0 0 0 4 0 4 0 4 0]
 [4 4 4 4 0 0 0 0 0 4 4 0 0 0 4 0 0 0 4 0 0 0 0 4 0]
 [4 4 4 0 0 0 0 4 4 4 4 0 4 4 0 4 4 0 4 0 0 4 

Execução do código para n = 6.

In [9]:
pontos = [x+1 for x in range(36**2)]
pontos = np.reshape(pontos,(36,36))
edges=createEdges(pontos)
grafo5=nx.Graph(edges)
nx.set_node_attributes(grafo5,0,'color')
ti = time.perf_counter()
ip_color2(grafo5,36)
tf = time.perf_counter()
print(converte(grafo5))
print(f"Time {tf - ti: 0.5f} seconds")

KeyboardInterrupt: ignored